![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [31]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import joblib

# Import any additional modules and start coding below

In [2]:
df = pd.read_csv('rental_info.csv')

In [3]:
df.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rental_date       15861 non-null  object 
 1   return_date       15861 non-null  object 
 2   amount            15861 non-null  float64
 3   release_year      15861 non-null  float64
 4   rental_rate       15861 non-null  float64
 5   length            15861 non-null  float64
 6   replacement_cost  15861 non-null  float64
 7   special_features  15861 non-null  object 
 8   NC-17             15861 non-null  int64  
 9   PG                15861 non-null  int64  
 10  PG-13             15861 non-null  int64  
 11  R                 15861 non-null  int64  
 12  amount_2          15861 non-null  float64
 13  length_2          15861 non-null  float64
 14  rental_rate_2     15861 non-null  float64
dtypes: float64(8), int64(4), object(3)
memory usage: 1.8+ MB


In [5]:
df['rental_date']= pd.to_datetime(df['rental_date'])
df['return_date']= pd.to_datetime(df['return_date'])

In [6]:
df.dtypes

rental_date         datetime64[ns, UTC]
return_date         datetime64[ns, UTC]
amount                          float64
release_year                    float64
rental_rate                     float64
length                          float64
replacement_cost                float64
special_features                 object
NC-17                             int64
PG                                int64
PG-13                             int64
R                                 int64
amount_2                        float64
length_2                        float64
rental_rate_2                   float64
dtype: object

In [7]:
df['rental_length_days'] = (df['return_date'] - df['rental_date']).dt.days
df['rental_length_days']

0        3
1        2
2        7
3        2
4        4
        ..
15856    6
15857    4
15858    9
15859    8
15860    6
Name: rental_length_days, Length: 15861, dtype: int64

In [14]:
df.special_features.value_counts()

special_features
{Trailers,Commentaries,"Behind the Scenes"}                     1308
{Trailers}                                                      1139
{Trailers,Commentaries}                                         1129
{Trailers,"Behind the Scenes"}                                  1122
{"Behind the Scenes"}                                           1108
{Commentaries,"Deleted Scenes","Behind the Scenes"}             1101
{Commentaries}                                                  1089
{Commentaries,"Behind the Scenes"}                              1078
{Trailers,"Deleted Scenes"}                                     1047
{"Deleted Scenes","Behind the Scenes"}                          1035
{"Deleted Scenes"}                                              1023
{Commentaries,"Deleted Scenes"}                                 1011
{Trailers,Commentaries,"Deleted Scenes","Behind the Scenes"}     983
{Trailers,Commentaries,"Deleted Scenes"}                         916
{Trailers,"Delete

In [13]:
df["deleted_scenes"] =  np.where(df["special_features"].str.contains("Deleted Scenes"), 1,0)

In [11]:
df['behind_the_scenes']= np.where(df['special_features'].str.contains('Behind the Scenes'),1,0)

In [15]:
corr_matrix =df.corr()
plt.figure(figsize=(12,6))
sns.heatmap(corr_matrix,annot= True,cmap ='coolwarm',linewidths=0.3)
plt.show()

ValueError: could not convert string to float: '{Trailers,"Behind the Scenes"}'

In [27]:
x = df.drop(['rental_length_days','return_date','rental_date','special_features'],axis=1)
y = df['rental_length_days']

In [28]:
x.columns

Index(['amount', 'release_year', 'rental_rate', 'length', 'replacement_cost',
       'NC_17', 'PG', 'PG_13', 'R', 'amount_2', 'length_2', 'rental_rate_2',
       'deleted_scenes', 'behind_the_scenes'],
      dtype='object')

In [21]:
x.dtypes

amount               float64
release_year         float64
rental_rate          float64
length               float64
replacement_cost     float64
NC-17                  int64
PG                     int64
PG-13                  int64
R                      int64
amount_2             float64
length_2             float64
rental_rate_2        float64
deleted_scenes         int64
behind_the_scenes      int64
dtype: object

In [26]:
df.rename(columns={"NC-17":"NC_17","PG-13":"PG_13"},inplace=True)

In [18]:
y.dtype

dtype('int64')

In [19]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=9)

In [29]:
lin = LinearRegression()
lin.fit(x_train,y_train)
y_pred = lin.predict(x_test)
mae_score = mean_squared_error(y_pred,y_test)
R2_Score = r2_score(y_pred,y_test)
print(mae_score)
print(R2_Score)



2.941723864697588
0.2943453518282221


In [30]:
rf_model = RandomForestRegressor()
rf_model.fit(x_train,y_train)
y_pred_rf = rf_model.predict(x_test)
mae_score_rf = mean_squared_error(y_pred_rf,y_test)
R2_score = r2_score(y_pred,y_test)
print(mae_score_rf)
print(R2_score)

2.0306525769171806
0.2943453518282221


In [34]:
joblib.dump(rf_model,'dvd_model.pkl')

['dvd_model.pkl']

In [42]:
x.columns

Index(['rental_date', 'return_date', 'amount', 'release_year', 'rental_rate',
       'length', 'replacement_cost', 'special_features', 'NC-17', 'PG',
       'PG-13', 'R', 'amount_2', 'length_2', 'rental_rate_2', 'deleted_scenes',
       'behind_the_scenes'],
      dtype='object')